<a href="https://colab.research.google.com/github/VinylBr/Project_MTBResistance/blob/main/Project_184026002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Vinay Barnabas

In [18]:
import pandas as pd
import numpy as np
import gdown #to download from gdrive

# Upload Datasets
----

In [7]:
!gdown https://drive.google.com/uc?id=1-46hUwDdUhE9DGBAkx9gPx6FTvOSiV-c #Download from google drive TEST_LEV.csv

Downloading...
From: https://drive.google.com/uc?id=1-46hUwDdUhE9DGBAkx9gPx6FTvOSiV-c
To: /content/TEST_LEV.csv
100% 86.8M/86.8M [00:00<00:00, 139MB/s]


In [14]:
!gdown --fuzzy https://docs.google.com/spreadsheets/d/1E3_rZWmTXyd-YE_oAfxZECW_bi-QoNFa/edit?usp=drive_link&ouid=106623698305590473663&rtpof=true&sd=true

Downloading...
From: https://drive.google.com/uc?id=1E3_rZWmTXyd-YE_oAfxZECW_bi-QoNFa
To: /content/Resistantstatus.xlsx
100% 392k/392k [00:00<00:00, 118MB/s]


# Command line: Ubuntu executed using windows subsystem for Linux (WSL)

zcat MUTATIONS.csv.gz | awk -F',' '{ if (NF > max_fields) max_fields = NF } END { print max_fields }' *print the maximum field* <br>

zcat MUTATIONS.csv.gz | cut -d ',' -f 2 | tail -n +2 | sort | uniq > genes.txt *get the list of genes* <br>
<br>
zcat DST_MEASUREMENTS.csv.gz | grep "LEV" | grep "CRyPTIC" > DST_LEV.csv <br>
*Get only Drug Susceptibility testing (DST) data for Levofloxacin(LEV) measured by CRyPTIC consortium. Assumption no. 1* <br>

cat DST_LEV.csv | awk -F "," '{print $1}' | uniq  > SAMPLEID.csv
*get only sample IDs where Levofloxacin resistance has been tested* <br>
<br>
zcat MUTATIONS.csv.gz | grep -Fw -f SAMPLEID.csv | gzip > MUTATIONS_redacted.csv.gz *reduce the Mutations dataset* <br>
zcat MUTATIONS_redacted.csv.gz | awk -F "," '{print $1","$2}' | uniq | awk -F',' 'NR>1{genes[$1] = genes[$1] "," $2} END {for (sample in genes) print sample","substr(genes[sample], 2)}' > DATAMATRIX_LEV.csv <br>
*takes only SAMPLEID and Gene names, gets unique values, creates a csv with rows as SAMPLEID and columns as genes*<br>
<br>
<br>
DATAMATRIX_LEV.csv has 11,183 samples <br>
Maximum number of mutated genes in a sample 1544. This is lower than the total number of genes which is around 3862 <br>

IN DST_measurement for LEV with CRyPTIC consortium dataset = 15,224 samples <br>
for LEV alone there were around 20,668 samples <br>
using LEV + CRyPTIC dataset <br>


In [19]:
df = pd.read_csv("TEST_LEV.csv")
Resistance_status_df = pd.read_excel("Resistantstatus.xlsx")

In [20]:
df.head()

,SAMPLEID,eccB1,eccB2,eccB3,Rv2331,eccB4,eccB5,Rv2499c,Rv2336,infA,...,Rv1836c,Rv1003,Rv2366c,Rv1006,Rv0963c,Rv3651,mutY,Rv1703c,bacA,Rv1882c
0,site.03.subj.GB-83750038.lab.83750038.iso.1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,site.06.subj.SGD_0014-14.lab.06MIL0081.iso.1,0,0,1,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,1,1
2,site.05.subj.LS-0833.lab.LS-10414-18.iso.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,site.20.subj.SCH7887477.lab.YA00134140.iso.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,site.02.subj.0455.lab.251040-15.iso.1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
Resistance_status_df.head()

,UNIQUEID,BDQ,AMI,CFZ,DLM,EMB,ETH,INH,KAN,LEV,LZD,MXF,RFB,RIF
0,site.06.subj.06TB_0277.lab.06MIL0868.iso.1,S_BDQ,S_AMI,S_CFZ,S_DLM,R_EMB,R_ETH,R_INH,R_KAN,R_LEV,S_LZD,R_MXF,R_RFB,R_RIF
1,site.04.subj.00830.lab.22543.iso.1,S_BDQ,S_AMI,S_CFZ,S_DLM,S_EMB,S_ETH,S_INH,S_KAN,R_LEV,S_LZD,S_MXF,S_RFB,S_RIF
2,site.02.subj.0057.lab.22A007.iso.1,S_BDQ,S_AMI,S_CFZ,S_DLM,R_EMB,R_ETH,R_INH,S_KAN,R_LEV,S_LZD,R_MXF,R_RFB,R_RIF
3,site.02.subj.1109.lab.2014185019.iso.1,S_BDQ,S_AMI,S_CFZ,S_DLM,S_EMB,S_ETH,S_INH,S_KAN,S_LEV,S_LZD,S_MXF,S_RFB,S_RIF
4,site.08.subj.02TB2334.lab.26078.iso.1,S_BDQ,S_AMI,S_CFZ,S_DLM,S_EMB,S_ETH,S_INH,S_KAN,S_LEV,S_LZD,S_MXF,S_RFB,S_RIF


In [46]:
df.columns = df.columns.str.replace("SAMPLEID", "UNIQUEID") #change to allow concat
df_alldrugs = pd.concat([df, Resistance_status_df], axis = 1, join = "inner") #concat dataframes with the column column UNIQUEID along axis 1 (columns), by keeping intersection of UNIQUEIDs

**Check

In [47]:
df_alldrugs.head()

,UNIQUEID,eccB1,eccB2,eccB3,Rv2331,eccB4,eccB5,Rv2499c,Rv2336,infA,...,DLM,EMB,ETH,INH,KAN,LEV,LZD,MXF,RFB,RIF
0,site.03.subj.GB-83750038.lab.83750038.iso.1,1,0,0,0,0,0,0,0,0,...,S_DLM,R_EMB,R_ETH,R_INH,R_KAN,R_LEV,S_LZD,R_MXF,R_RFB,R_RIF
1,site.06.subj.SGD_0014-14.lab.06MIL0081.iso.1,0,0,1,0,1,1,0,1,0,...,S_DLM,S_EMB,S_ETH,S_INH,S_KAN,R_LEV,S_LZD,S_MXF,S_RFB,S_RIF
2,site.05.subj.LS-0833.lab.LS-10414-18.iso.1,0,0,0,0,0,0,0,0,0,...,S_DLM,R_EMB,R_ETH,R_INH,S_KAN,R_LEV,S_LZD,R_MXF,R_RFB,R_RIF
3,site.20.subj.SCH7887477.lab.YA00134140.iso.1,0,0,0,0,0,0,0,0,0,...,S_DLM,S_EMB,S_ETH,S_INH,S_KAN,S_LEV,S_LZD,S_MXF,S_RFB,S_RIF
4,site.02.subj.0455.lab.251040-15.iso.1,0,0,0,0,1,0,0,0,0,...,S_DLM,S_EMB,S_ETH,S_INH,S_KAN,S_LEV,S_LZD,S_MXF,S_RFB,S_RIF


In [49]:
df_alldrugs.iloc[1,df_alldrugs.loc[0].values==1].head(n = 5) #cross-checked with the original datafile to ensure datamanipulation did not introduce errors

eccB1      0
Rv0008c    1
Rv0575c    1
Rv0964c    1
Rv3669     1
Name: 1, dtype: object

In [57]:
df_alldrugs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6197 entries, 0 to 6196
Columns: 3876 entries, UNIQUEID to RIF
dtypes: int64(3861), object(15)
memory usage: 183.3+ MB


**Observations**: <br>
All genes have integer entries <br>
Resistance information (14 drugs) and UNIQUEID are the objects, totally 5 columns

In [50]:
df_alldrugs.describe()

,eccB1,eccB2,eccB3,Rv2331,eccB4,eccB5,Rv2499c,Rv2336,infA,infB,...,Rv1836c,Rv1003,Rv2366c,Rv1006,Rv0963c,Rv3651,mutY,Rv1703c,bacA,Rv1882c
count,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000,...,6197.000000,6197.000000,6197.000000,6197.00000,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000,6197.000000
mean,0.197354,0.082137,0.092303,0.043892,0.501694,0.071648,0.015491,0.135388,0.016137,0.209940,...,0.142972,0.024205,0.324996,0.04599,0.046797,0.133613,0.083912,0.024044,0.112635,0.047765
std,0.398034,0.274595,0.289476,0.204872,0.500037,0.257924,0.123506,0.342165,0.126012,0.407299,...,0.350073,0.153698,0.468411,0.20948,0.211220,0.340264,0.277278,0.153198,0.316172,0.213286
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


**Observation**:
Data has ones and zeros <br>
Some genes have mutations in 50% of the isolates like eccB4
